In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/simonpernegger/Library/CloudStorage/OneDrive-Persönlich/UNI/2_CSE/KTH_deeplearning/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-07 14:21:40,886 INFO: Initializing external client
2025-11-07 14:21:40,888 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-07 14:21:53,760 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1271993


'{"country": "austria", "city": "vienna", "street": "schottenfeldgasse", "aqicn_url": "https://api.waqi.info/feed/A56884", "latitude": "48.200458", "longitude": "16.343257"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [11]:
import requests
import pandas as pd
import numpy as np

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df["pm25"] = aq_today_df["pm25"].astype(np.double)

In [12]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [13]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 48.25°N 16.25°E
Elevation 219.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-07,11.35,0.0,22.771244,129.225662,vienna
1,2025-11-08,11.25,0.0,9.793058,107.102814,vienna
2,2025-11-09,9.95,0.0,17.873556,279.272522,vienna
3,2025-11-10,10.35,0.0,9.107359,288.435028,vienna
4,2025-11-11,10.75,0.0,9.605998,282.994629,vienna
5,2025-11-12,11.35,0.0,1.800000,360.000000,vienna
6,2025-11-13,7.25,0.0,16.055355,132.273636,vienna


In [14]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [15]:
# Insert new data
air_quality_fg.insert(aq_today_df)

%6|1762521807.120|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 49914ms in state UP)
%6|1762521807.849|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50175ms in state UP)
%6|1762521811.144|FAIL|rdkafka#consumer-2| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50135ms in state UP)
%6|1762521816.900|FAIL|rdkafka#consumer-2| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 58118ms in state UP)
%6|1762521817.164|FAIL|rdkafka#consumer-2| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 58382ms in state UP)
%6|1762521820.934|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 52895ms in state UP)
Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remai

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271993/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'), None)

In [ ]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

%6|1762521868.838|FAIL|rdkafka#consumer-2| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 105139ms in state UP)
%6|1762521871.735|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50135ms in state UP, 1 identical error(s) suppressed)
Uploading Dataframe: 0.00% |          | Rows 0/7 | Elapsed Time: 00:00 | Remaining Time: ?%6|1762521879.618|FAIL|rdkafka#consumer-2| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 53052ms in state UP, 1 identical error(s) suppressed)
Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271993/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-07 14:24:49,333 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-07 14:24:52,522 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


%6|1762521923.927|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50127ms in state UP, 1 identical error(s) suppressed)
%6|1762521925.941|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50703ms in state UP, 1 identical error(s) suppressed)
%6|1762521930.999|FAIL|rdkafka#consumer-2| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50227ms in state UP, 1 identical error(s) suppressed)
%6|1762521933.271|FAIL|rdkafka#consumer-2| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50650ms in state UP, 1 identical error(s) suppressed)
%6|1762521978.113|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50116ms in state UP, 1 identical error(s) suppressed)
%6|1762521980.894|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.81.208

2025-11-07 14:27:35,442 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-07 14:27:35,628 INFO: Waiting for log aggregation to finish.
2025-11-07 14:27:44,283 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'), None)

%6|1762522084.587|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50131ms in state UP, 1 identical error(s) suppressed)
%6|1762522085.363|FAIL|rdkafka#consumer-2| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50249ms in state UP, 1 identical error(s) suppressed)
%6|1762522085.546|FAIL|rdkafka#consumer-2| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50129ms in state UP, 1 identical error(s) suppressed)
%6|1762522105.522|FAIL|rdkafka#producer-1| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 70200ms in state UP, 1 identical error(s) suppressed)
%6|1762522135.503|FAIL|rdkafka#consumer-2| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 100289ms in state UP, 1 identical error(s) suppressed)
%6|1762522136.001|FAIL|rdkafka#consumer-2| [thrd:GroupCoordinator]:

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
